<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/telematics_syn_V11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch_tabnet

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
import torch

# Load the dataset from Google Drive
file_path = '/content/drive/My Drive/telematics_syn.csv'
df = pd.read_csv(file_path)

In [ ]:
import pandas as pd
import numpy as np

# Create a binary Risk Category label
df['Risk_Category'] = np.where(df['NB_Claim'] == 0, 'Low Risk', 'High Risk')

# Save the updated dataset
df.to_csv('telematics_syn_updated.csv', index=False)

# Display the first few rows to verify the new column
print(df.head())

In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder

# Load your dataset
data = pd.read_csv('telematics_syn_updated.csv')

# Separate features and target variable
X = data.drop('Risk_Category', axis=1)
y = data['Risk_Category']

# Identify columns with non-numerical data
categorical_cols = X.select_dtypes(include=['object']).columns

# Apply label encoding to categorical features
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le  # Store the encoders for later use if needed

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Convert resampled data to a DataFrame
balanced_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['Risk_Category'])], axis=1)

# Check the distribution of the target variable
print(Counter(balanced_data['Risk_Category']))

# Save the balanced dataset
balanced_data.to_csv('balanced_dataset.csv', index=False)
output:
Counter({'High Risk': 95728, 'Low Risk': 95728})
# Count the occurrences of 1's and 0's in Risk_Category
claim_yn_counts = df['Risk_Category'].value_counts()

print(claim_yn_counts)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the balanced dataset
balanced_data = pd.read_csv('balanced_dataset.csv')

# Identify categorical and numerical columns
categorical_cols = ['Insured.sex', 'Marital', 'Car.use', 'Region', 'Territory']
numerical_cols = [col for col in balanced_data.columns if col not in categorical_cols + ['Risk_Category']]

# Preprocess the data: one-hot encode categorical variables and normalize numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Separate features and labels
X = balanced_data.drop('Risk_Category', axis=1)
y = balanced_data['Risk_Category']

# Apply preprocessing
X_preprocessed = preprocessor.fit_transform(X)

# Split the data into training, validation, and testing sets (60% train, 20% validation, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X_preprocessed, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Check the shape of the splits to ensure they are correct
print(f'Training set shape: {X_train.shape}, {y_train.shape}')
print(f'Validation set shape: {X_val.shape}, {y_val.shape}')
print(f'Testing set shape: {X_test.shape}, {y_test.shape}')

# Save the preprocessed data to CSV files if needed
train_data = pd.DataFrame(X_train, columns=preprocessor.get_feature_names_out())
train_data['Risk_Category'] = y_train.values
train_data.to_csv('train_data.csv', index=False)

val_data = pd.DataFrame(X_val, columns=preprocessor.get_feature_names_out())
val_data['Risk_Category'] = y_val.values
val_data.to_csv('val_data.csv', index=False)

test_data = pd.DataFrame(X_test, columns=preprocessor.get_feature_names_out())
test_data['Risk_Category'] = y_test.values
test_data.to_csv('test_data.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch

# Load the dataset
df = pd.read_csv('telematics_syn_updated.csv')

# Create Risk_Category based on the conditions
df['Risk_Category'] = np.where((df['NB_Claim'] > 1) & (df['AMT_Claim'] > 1000), 1, 0)

# Identify categorical and numerical columns
categorical_cols = ['Insured.sex', 'Marital', 'Car.use', 'Region', 'Territory']
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['Risk_Category']]

# Preprocess the data: one-hot encode categorical variables and normalize numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Separate features and labels
X = df.drop('Risk_Category', axis=1)
y = df['Risk_Category']

# Apply preprocessing
X_preprocessed = preprocessor.fit_transform(X)

# Split the data into training, validation, and testing sets (60% train, 20% validation, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X_preprocessed, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Initialize TabNet model
tabnet_model = TabNetClassifier(
    n_d=64, n_a=64, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
    lambda_sparse=1e-3, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type="entmax",
    scheduler_params=dict(mode="min", patience=5, min_lr=1e-5, factor=0.9),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    seed=42,
    verbose=10
)

# Train the model
tabnet_model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    max_epochs=100,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

# Evaluate the model on the validation set
y_val_pred = tabnet_model.predict(X_val)

# Calculate accuracy and F1 score
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred, average='weighted')

print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")

# Save the model
saving_path_name = "tabnet_model"
saved_filepath = tabnet_model.save_model(saving_path_name)
print(f"Model saved to: {saved_filepath}")


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.05499 | train_accuracy: 0.99823 | valid_accuracy: 0.99815 |  0:00:34s
epoch 10 | loss: 0.00198 | train_accuracy: 0.99985 | valid_accuracy: 0.99965 |  0:05:51s
epoch 20 | loss: 0.00102 | train_accuracy: 0.99997 | valid_accuracy: 0.99975 |  0:11:07s

Early stopping occurred at epoch 26 with best_epoch = 16 and best_valid_accuracy = 0.99975


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Validation Accuracy: 0.9998
Validation F1 Score: 0.9998
Successfully saved model at tabnet_model.zip
Model saved to: tabnet_model.zip
